In [3]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

features=['draw','addedWeight','declaredHorseWeight','horse_winPer',
                 'horse_winCount','jockey_winPer','jockey_winCount','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace']
target=['finishTime']
file0 = "/Users/apple/Workspace/intelliJ/jockey/history.csv"

file1 = "/Users/apple/Workspace/intelliJ/jockey/predict.csv"
tableName="newrace"
#file1 = "/Users/apple/Workspace/intelliJ/jockey/regress.csv"
#tableName="racecard"

dataset0 = pd.read_csv(file0)
dataset1 = pd.read_csv(file1)

In [4]:
allDates = dataset1.rateDate.unique()
allDates.sort()
for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset1[(dataset1['rateDate']==raceDate)]
    allSeqs = dataset_ofADay.raceSeqOfDate.unique()
    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset1['raceSeqOfDate']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(seq,distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['rateDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
       


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #      % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))

        #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=X_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['rateDate','raceSeqOfDate','distance','raceClass','horseId']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #dataset_test_withPredicted

        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['rateDate'])
            raceSeqOfDate =str(row['raceSeqOfDate'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.fetchall(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDate'] ,",", row['horseId'] )

20190213
4 1650 4
1
1  record(s) updated 4 , B102
2
1  record(s) updated 4 , A313
3
1  record(s) updated 4 , T326
4
1  record(s) updated 4 , B358
5
1  record(s) updated 4 , V378
6
1  record(s) updated 4 , T390
7
1  record(s) updated 4 , T055
8
1  record(s) updated 4 , B386
9
1  record(s) updated 4 , T369
10
1  record(s) updated 4 , A399
11
1  record(s) updated 4 , T338
12
1  record(s) updated 4 , A366
1 1000 4
1
1  record(s) updated 1 , A254
2
1  record(s) updated 1 , A408
3
1  record(s) updated 1 , C128
4
1  record(s) updated 1 , C218
5
1  record(s) updated 1 , P429
6
1  record(s) updated 1 , B417
7
1  record(s) updated 1 , C014
8
1  record(s) updated 1 , C172
9
1  record(s) updated 1 , S279
10
1  record(s) updated 1 , C097
11
1  record(s) updated 1 , V123
12
1  record(s) updated 1 , B445
2 1650 5
1
1  record(s) updated 2 , S417
2
1  record(s) updated 2 , T289
3
1  record(s) updated 2 , A191
4
1  record(s) updated 2 , A334
5
1  record(s) updated 2 , T022
6
1  record(s) updated 2 , A38